In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pyodbc
import time

def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [9]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time
import re
import os
import csv

#Acesso ao banco de dados Casa
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server inexistente ou acesso negado. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

In [11]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:01<00:00, 4.62MB/s]
C:\Users\GLEISS~1.SAN\AppData\Local\Temp/ipykernel_17764/3643764433.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [12]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [13]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(3)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'detalhe', 'DOSE_anchor')

####  Acesso ao item "Detalhes da Os"

In [14]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-panel"]/input').click()
    #coletar valor solicitado
    
    try:
        material = driver.find_element(By.CSS_SELECTOR, "label[title='Material Utilizado'] ").text
    except NoSuchElementException:
        material = 'None'
    
    try:
        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD'] ").text
    except NoSuchElementException:
        try:
            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD Instalado'] ").text
        except NoSuchElementException:
            try:
                hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro Instalado'] ").text
            except NoSuchElementException:
                try:
                    hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro'] ").text
                except NoSuchElementException:
                    try:
                        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Hidrômetro Instalado'] ").text
                    except NoSuchElementException:
                        try:
                            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro para Religação'] ").text
                        except NoSuchElementException:
                            hd = None
                        
    return material, hd


In [17]:
df = pd.read_sql('SELECT * FROM [HDs_Util 26 a 25.03_LF]', executar_conexao_bd())
#df = pd.DataFrame(df, columns = ['SS', 'OS','Serviço', 'Material', 'Mureta','Hd'])
df = pd.DataFrame(df, columns = ['SS', 'OS','Serviço','Hd'])
#df = df.astype(str)

#df = df.query("Tipo == '21 - LIGACAO DE AGUA'")

display(df)
n_linhas = len(df)
print("Número de linhas: ", n_linhas)

Executado a conexão com o BD......


,SS,OS,Serviço,Hd
0,980905781,2,12 - SUBSTITUICAO HID.,NaN
1,980905783,2,12 - SUBSTITUICAO HID.,NaN
2,980905789,2,12 - SUBSTITUICAO HID.,NaN
3,980905808,2,12 - SUBSTITUICAO HID.,NaN
4,980905790,2,12 - SUBSTITUICAO HID.,NaN
...,...,...,...,...
579,977892673,1,672 - SUBSTITUICAO HID COD 34,NaN
580,980496103,1,672 - SUBSTITUICAO HID COD 34,NaN
581,980540667,1,672 - SUBSTITUICAO HID COD 34,NaN
582,980540668,1,672 - SUBSTITUICAO HID COD 34,NaN


Número de linhas:  584


In [18]:
#Extrair valores e criar coluna Material insumado
dados = []
hds = []
for i, row in df.iterrows():
    a = preencher_ss(driver, row['SS'], row['OS'])
    time.sleep(0.3)
    dados.append(a[0])
    hds.append(a[1])
df['Material'] = dados
df['Hd'] = hds


In [19]:
display(df)

,SS,OS,Serviço,Hd,Material
0,980905781,2,12 - SUBSTITUICAO HID.,A22HW0293034,
1,980905783,2,12 - SUBSTITUICAO HID.,A22SG0043829,
2,980905789,2,12 - SUBSTITUICAO HID.,A22SG0043827,
3,980905808,2,12 - SUBSTITUICAO HID.,A22SG0031070,
4,980905790,2,12 - SUBSTITUICAO HID.,A22SG0043821,
...,...,...,...,...,...
579,977892673,1,672 - SUBSTITUICAO HID COD 34,A22SG0072613,1 MURETA 10 METROS DE TUBO AZUL PEAD DN 20MM. ...
580,980496103,1,672 - SUBSTITUICAO HID COD 34,A22SG0072394,1 CAPSULAME PVC DN 3/4 1 BUCHA DE REDUÇÃO PVC ...
581,980540667,1,672 - SUBSTITUICAO HID COD 34,A22SG0072392,1 HIDRO VZ 3M. 1 LACRE AZUL FIO DE AÇO 1 CAPSU...
582,980540668,1,672 - SUBSTITUICAO HID COD 34,A22SG0072393,1 HIDRO VZ 3M. 1 LACRE AZUL FIO DE AÇO 1 CAPSU...


In [20]:
#Acessar tabela do BD e extrair apenas item do texto
m_encontrado = ['Mureta', 'MURETA','mureta']

material = []

for i, row in df.iterrows():
    if any(x in row['Material'] for x in m_encontrado):
        material.append('Sim')
    else:
        material.append('Não')     
df['Mureta'] = material


df = df.astype(str)
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
df_sim = df.loc[df['Mureta'] == 'Sim']
display(df_sim)


,SS,OS,Material,Mureta,Hd
134,981202578,2,1 MURETA 1 HIDRO VZ 3M. 1 LACRE AZUL FIO DE AÇ...,Sim,A22SG0072397
223,981944686,2,1 METRO TUBO PEAD 20MM 1 ADAPTADOR MACHO PEAD ...,Sim,A22HW0180315
524,981187654,2,1 colar 150 5 metros de tubo pead 1 adaptador ...,Sim,A22HW0030303
525,981484353,1,1 colar de 85 1 adaptador pead 8 metros de tub...,Sim,A22HW0291504
554,981115629,2,3 METROS DE TUBO AZUL PEAD DN 20MM. 1 COLAT DE...,Sim,A22HW0293290
555,981115992,2,5 METROS DE TUBO AZUL PEAD DN 20MM. 1 COLAT DE...,Sim,A22HW0293289
556,981172632,2,3 METROS DE TUBO AZUL PEAD DN 20MM. 1 COLAT DE...,Sim,A22HW0293282
557,981174726,2,Mureta com tampa 1 registro angular 20×1/2 1hi...,Sim,A22HW0293497
558,981177171,2,1 mureta com tampa 1joelho pead com rosa 20×1/...,Sim,A22HW0291342
559,981180598,2,3 METROS DE TUBO AZUL PEAD DN 20MM. 1 COLAT DE...,Sim,A22HW0293281


In [21]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    #comando_sql = "UPDATE [700_Lauro] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    #ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    comando_sql = "UPDATE [HDs_Util 26 a 25.03_LF] SET Hd='{}' WHERE SS={}".format(
    HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [2]:
#Limpar colunas do Banco de dados;
#delete_query = f"DELETE FROM {table_name}"
#df = df.astype(str)
#df = pd.DataFrame(df, columns=['SS', 'OS', 'Matutilizado', 'Hdutilizado', 'Usoumureta'])

#cursor = executar_conexao_bd().cursor()
#for index, row in df.iterrows():
    #Mat = row.Matutilizado
    #HD  = row.Hdutilizado
    #INS = row.Usoumureta
    #SS  = row.SS
    #comando_sql = (f"UPDATE Material SET Matutilizado='NULL', Hdutilizado='NULL', Usoumureta='NULL' WHERE SS ={SS}")
    #cursor.execute(comando_sql)
    
#cursor.commit()
#cursor.close()